## Importaciones necesarias

In [1]:
using JuMP, Ipopt, Complementarity

# Crear el modelo

In [2]:
# Crear el modelo
model = Model(Ipopt.Optimizer)

A JuMP Model
├ solver: Ipopt
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

## Definir Variables


In [3]:
# Nivel superior
@variable(model,x)
# Nivel Inferior
@variable(model,y)
# Complementariedad
@variable(model,l[1:3])

3-element Vector{VariableRef}:
 l[1]
 l[2]
 l[3]

## Definir Objetivo


In [4]:
@objective(model,Min,x^2)

x²

## Definir Restricciones


In [5]:
@NLconstraint(model,1+(-0.98l[1]+4.75l[2]+(l[3]*(2*y*(x-0.25))+3.01l[3]))==0) # Obj nivel inferior
@NLconstraint(model,-1.19 + 3.01y + (-0.5 + x)*(y^2))<=0 )#vs

AssertionError: AssertionError: expr.args[1] == :(==)

In [6]:
@constraints(model,begin
2.89 - x<=0  #gs
-2.89 + x<=0  #gs
72.5789 + x - y - 9(x^2)<=0 #gs
0.29 - 0.98y<=0  #vs
-2.19 + 4.75y<=0 #vs

end)

(-x <= -2.89, x <= 2.89, -9 x² + x - y <= -72.5789, -0.98 y <= -0.29, 4.75 y <= 2.19)

## Definir variables Complementarias

In [7]:
@complements(model,0<=-(0.29 - 0.98),l[1]>=0)
@complements(model,0<=-(-2.19 + 4.75y),l[2]>=0)
@complements(model,0<=-(-1.19 + 3.01y + (-0.5 + x)*(y^2)),l[3]>=0)

(sqrt(l[3] ^ 2.0 + (-(((-1.19 + 3.01 * y) + (-0.5 + x) * (y * y))) - 0.0) ^ 2.0 + 1.0e-8) - (l[3] + (-(((-1.19 + 3.01 * y) + (-0.5 + x) * (y * y))) - 0.0))) - 0.0 == 0

## Optimizar

In [8]:
# Resolver el modelo
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:       11
Number of nonzeros in inequality constraint Jacobian.:       13
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        5
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        4
Total number of inequality c

# Mostrar resultados

In [9]:
# Mostrar resultados
println("Estado de la solución: ", termination_status(model))
println("Valor óptimo de x: ", value(x))
println("Valor óptimo de y1: ", value(y))
#println("Valor óptimo de y2: ", value(y2))
#println("Valores óptimos de s: ", value.(s))
println("Valores óptimos de l: ", value.(l))

Estado de la solución: LOCALLY_INFEASIBLE
Valor óptimo de x: 2.8900799854866612
Valor óptimo de y1: 0.2959183571489294
Valores óptimos de l: [7.247447833190114e-9, -0.19655278627200617, -0.014515978640721506]


In [10]:
objective_value(model)

8.35256232251058